In [2]:
import tensorflow.keras.layers as Layers
#import tensorflow.keras.activations as Actications
import tensorflow.keras.models as Models
import tensorflow.keras.optimizers as Optimizer
import tensorflow.keras.metrics as Metrics
import tensorflow.keras.utils as Utils
import tensorflow.keras.callbacks as Callbacks
#from tensorflow.python.keras.utils.vis_utils import model_to_dot
import os
#import matplotlib.pyplot as plot
import cv2
import numpy as np
from sklearn.utils import shuffle
#from sklearn.metrics import confusion_matrix as CM
#from random import randint
#from IPython.display import SVG

In [7]:
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [8]:
def get_images(directory):
    Images = []
    Labels = []  # 0 for Building , 1 for forest, 2 for glacier, 3 for mountain, 4 for Sea , 5 for Street
    label = 0
    
    for labels in os.listdir(directory): #Main Directory where each class label is present as folder name.
        if labels == 'glacier': #Folder contain Glacier Images get the '2' class label.
            label = 2
        elif labels == 'sea':
            label = 4
        elif labels == 'buildings':
            label = 0
        elif labels == 'forest':
            label = 1
        elif labels == 'street':
            label = 5
        elif labels == 'mountain':
            label = 3
        
        for image_file in os.listdir(directory+labels): #Extracting the file name of the image from Class Label folder
            image = cv2.imread(directory+labels+r'/'+image_file) #Reading the image (OpenCV)
            image = cv2.resize(image,(150,150)) #Resize the image, Some images are different sizes. (Resizing is very Important)
            Images.append(image)
            Labels.append(label)

    return shuffle(Images,Labels,random_state=817328462) #Shuffle the dataset you just prepared.

def get_classlabel(class_code):
    labels = {2:'glacier', 4:'sea', 0:'buildings', 1:'forest', 5:'street', 3:'mountain'}
    
    return labels[class_code]

In [9]:
Images, Labels = get_images('../archive/seg_train/seg_train/') #Extract the training images from the folders.

Images = np.array(Images) #converting the list of images to numpy array.
Labels = np.array(Labels)

In [18]:
# test_images, test_labels = get_images('../archive/seg_test/seg_test/')
# test_images = np.array(test_images)
# test_labels = np.array(test_labels)

In [10]:
#get current working directory path + log dir
root_logdir = os.path.join(os.curdir, "my_logs")

#returns a file path of the ending in the current time down to seconds
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

In [36]:
model_1 = Models.Sequential()

model_1.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu',input_shape=(150,150,3), padding="same"))
model_1.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_1.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_1.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_1.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_1.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_1.add(Layers.Flatten())

model_1.add(Layers.Dense(64,activation='relu'))
model.add(Layers.Dropout(rate=0.5))
model_1.add(Layers.Dense(32,activation='relu'))
model.add(Layers.Dropout(rate=0.5))
model_1.add(Layers.Dense(6,activation='softmax'))

model_1.compile(optimizer=Optimizer.Adam(lr=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model_1.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_74 (Conv2D)           (None, 150, 150, 16)      448       
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_75 (Conv2D)           (None, 75, 75, 32)        4640      
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 38, 38, 32)        0         
_________________________________________________________________
conv2d_76 (Conv2D)           (None, 38, 38, 64)        18496     
_________________________________________________________________
max_pooling2d_48 (MaxPooling (None, 19, 19, 64)        0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 23104)           

In [37]:
run_logdir = get_run_logdir()
tensorboard_cb = Callbacks.TensorBoard(run_logdir)

history = model_1.fit(Images, Labels, 
                    batch_size=32, 
                    epochs=50, 
                    validation_split = 0.2,
                    callbacks=[tensorboard_cb])

Train on 11227 samples, validate on 2807 samples
Epoch 1/25
11227/11227 [==============================] - 5s 446us/sample - loss: 3.2204 - accuracy: 0.5103 - val_loss: 1.3240 - val_accuracy: 0.6003
Epoch 2/25
11227/11227 [==============================] - 4s 389us/sample - loss: 1.0719 - accuracy: 0.6478 - val_loss: 1.0201 - val_accuracy: 0.6534
Epoch 3/25
 4768/11227 [===========>..................] - ETA: 2s - loss: 0.7447 - accuracy: 0.7365

KeyboardInterrupt: 

In [ ]:
model_1.save('saved_model\\3x1_1mil_Relu')

In [46]:
model_2 = Models.Sequential()

model_2.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu',input_shape=(150,150,3), padding="same"))
model_2.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu', padding="same"))
model_2.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_2.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_2.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_2.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_2.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_2.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_2.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_2.add(Layers.Flatten())

model_2.add(Layers.Dense(64,activation='relu'))
model_2.add(Layers.Dropout(rate=0.5))
model_2.add(Layers.Dense(32,activation='relu'))
model.add(Layers.Dropout(rate=0.5))
model_2.add(Layers.Dense(6,activation='softmax'))

model_2.compile(optimizer=Optimizer.Adam(lr=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model_2.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_94 (Conv2D)           (None, 150, 150, 16)      448       
_________________________________________________________________
max_pooling2d_61 (MaxPooling (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_95 (Conv2D)           (None, 75, 75, 32)        4640      
_________________________________________________________________
conv2d_96 (Conv2D)           (None, 75, 75, 32)        9248      
_________________________________________________________________
max_pooling2d_62 (MaxPooling (None, 38, 38, 32)        0         
_________________________________________________________________
conv2d_97 (Conv2D)           (None, 38, 38, 64)        18496     
_________________________________________________________________
conv2d_98 (Conv2D)           (None, 38, 38, 64)      

In [47]:
run_logdir = get_run_logdir()
tensorboard_cb = Callbacks.TensorBoard(run_logdir)

history2 = model_2.fit(Images, Labels, 
                    batch_size=32, 
                    epochs=50, 
                    validation_split = 0.2,
                    callbacks=[tensorboard_cb])

Train on 11227 samples, validate on 2807 samples
Epoch 1/25
11227/11227 [==============================] - 6s 535us/sample - loss: 1.6988 - accuracy: 0.4226 - val_loss: 1.0286 - val_accuracy: 0.6152
Epoch 2/25
11227/11227 [==============================] - 5s 465us/sample - loss: 1.1183 - accuracy: 0.5585 - val_loss: 0.9028 - val_accuracy: 0.6879
Epoch 3/25
11227/11227 [==============================] - 5s 424us/sample - loss: 1.0008 - accuracy: 0.5973 - val_loss: 0.8804 - val_accuracy: 0.6633
Epoch 4/25
11227/11227 [==============================] - 5s 462us/sample - loss: 0.9064 - accuracy: 0.6438 - val_loss: 0.7372 - val_accuracy: 0.7357
Epoch 5/25
11227/11227 [==============================] - 5s 445us/sample - loss: 0.8262 - accuracy: 0.6765 - val_loss: 0.6952 - val_accuracy: 0.7592
Epoch 6/25
11227/11227 [==============================] - 5s 421us/sample - loss: 0.7785 - accuracy: 0.6963 - val_loss: 0.6375 - val_accuracy: 0.7627
Epoch 7/25
11227/11227 [===========================

In [19]:
model_2.save('saved_model\\3x2_1mil_Relu')

3000/3000 [==============================] - 0s 166us/sample - loss: 1.4399 - accuracy: 0.7380


[1.4399294182459514, 0.738]

In [49]:
model_3 = Models.Sequential()

model_3.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu',input_shape=(150,150,3), padding="same"))
model_3.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu', padding="same"))
model_3.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu', padding="same"))
model_3.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_3.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_3.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_3.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_3.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_3.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_3.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_3.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_3.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_3.add(Layers.Flatten())

model_3.add(Layers.Dense(64,activation='relu'))
model_3.add(Layers.Dropout(rate=0.5))
model_3.add(Layers.Dense(32,activation='relu'))
model_3.add(Layers.Dropout(rate=0.5))
model_3.add(Layers.Dense(6,activation='softmax'))

model_3.compile(optimizer=Optimizer.Adam(lr=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model_3.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_99 (Conv2D)           (None, 150, 150, 16)      448       
_________________________________________________________________
conv2d_100 (Conv2D)          (None, 150, 150, 16)      2320      
_________________________________________________________________
conv2d_101 (Conv2D)          (None, 150, 150, 16)      2320      
_________________________________________________________________
max_pooling2d_64 (MaxPooling (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_102 (Conv2D)          (None, 75, 75, 32)        4640      
_________________________________________________________________
conv2d_103 (Conv2D)          (None, 75, 75, 32)        9248      
_________________________________________________________________
conv2d_104 (Conv2D)          (None, 75, 75, 32)      

In [50]:
run_logdir = get_run_logdir()
tensorboard_cb = Callbacks.TensorBoard(run_logdir)

history2 = model_3.fit(Images, Labels, 
                    batch_size=32, 
                    epochs=100, 
                    validation_split = 0.2,
                    callbacks=[tensorboard_cb])

Train on 11227 samples, validate on 2807 samples
Epoch 1/25
11227/11227 [==============================] - 13s 1ms/sample - loss: 1.4490 - accuracy: 0.4057 - val_loss: 0.9872 - val_accuracy: 0.6138
Epoch 2/25
11227/11227 [==============================] - 10s 925us/sample - loss: 1.0620 - accuracy: 0.5621 - val_loss: 0.9468 - val_accuracy: 0.6345
Epoch 3/25
11227/11227 [==============================] - 10s 911us/sample - loss: 0.9592 - accuracy: 0.6068 - val_loss: 0.8189 - val_accuracy: 0.6651
Epoch 4/25
11227/11227 [==============================] - 11s 950us/sample - loss: 0.8878 - accuracy: 0.6319 - val_loss: 0.7317 - val_accuracy: 0.7282
Epoch 5/25
11227/11227 [==============================] - 12s 1ms/sample - loss: 0.8284 - accuracy: 0.6593 - val_loss: 0.6970 - val_accuracy: 0.7357
Epoch 6/25
11227/11227 [==============================] - 12s 1ms/sample - loss: 0.7892 - accuracy: 0.6803 - val_loss: 0.7331 - val_accuracy: 0.7211
Epoch 7/25
11227/11227 [===========================

In [ ]:
model_3.save('saved_model\\3x3_1mil_Relu')

In [53]:
model_4 = Models.Sequential()

model_4.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu',input_shape=(150,150,3), padding="same"))
model_4.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu', padding="same"))
model_4.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_4.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_4.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_4.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_4.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_4.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_4.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_4.add(Layers.Conv2D(128,kernel_size=(3,3),activation='relu', padding="same"))
model_4.add(Layers.Conv2D(128,kernel_size=(3,3),activation='relu', padding="same"))
model_4.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))


model_4.add(Layers.Flatten())
model_4.add(Layers.Dense(128,activation='relu'))
model_4.add(Layers.Dropout(rate=0.5))
model_4.add(Layers.Dense(64,activation='relu'))
model_4.add(Layers.Dropout(rate=0.5))
model_4.add(Layers.Dense(32,activation='relu'))
model.add(Layers.Dropout(rate=0.5))
model_4.add(Layers.Dense(6,activation='softmax'))

model_4.compile(optimizer=Optimizer.Adam(lr=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model_4.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_116 (Conv2D)          (None, 150, 150, 16)      448       
_________________________________________________________________
conv2d_117 (Conv2D)          (None, 150, 150, 16)      2320      
_________________________________________________________________
max_pooling2d_71 (MaxPooling (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_118 (Conv2D)          (None, 75, 75, 32)        4640      
_________________________________________________________________
conv2d_119 (Conv2D)          (None, 75, 75, 32)        9248      
_________________________________________________________________
max_pooling2d_72 (MaxPooling (None, 38, 38, 32)        0         
_________________________________________________________________
conv2d_120 (Conv2D)          (None, 38, 38, 64)      

In [54]:
run_logdir = get_run_logdir()
tensorboard_cb = Callbacks.TensorBoard(run_logdir)

history2 = model_4.fit(Images, Labels, 
                    batch_size=32, 
                    epochs=100, 
                    validation_split = 0.2,
                    callbacks=[tensorboard_cb])

Train on 11227 samples, validate on 2807 samples
Epoch 1/25
11227/11227 [==============================] - 9s 824us/sample - loss: 1.5106 - accuracy: 0.3622 - val_loss: 1.0676 - val_accuracy: 0.5611
Epoch 2/25
11227/11227 [==============================] - 8s 668us/sample - loss: 1.1093 - accuracy: 0.5477 - val_loss: 0.9410 - val_accuracy: 0.6323
Epoch 3/25
11227/11227 [==============================] - 7s 635us/sample - loss: 0.9759 - accuracy: 0.6011 - val_loss: 0.8041 - val_accuracy: 0.6826
Epoch 4/25
11227/11227 [==============================] - 7s 639us/sample - loss: 0.8885 - accuracy: 0.6540 - val_loss: 0.7745 - val_accuracy: 0.6751
Epoch 5/25
11227/11227 [==============================] - 8s 698us/sample - loss: 0.8156 - accuracy: 0.6882 - val_loss: 0.6820 - val_accuracy: 0.7399
Epoch 6/25
11227/11227 [==============================] - 8s 724us/sample - loss: 0.7784 - accuracy: 0.7019 - val_loss: 0.7370 - val_accuracy: 0.7221
Epoch 7/25
11227/11227 [===========================

In [ ]:
model_4.save('saved_model\\4x2_2mil_Relu')

In [55]:
model_5 = Models.Sequential()

model_5.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu',input_shape=(150,150,3), padding="same"))
model_5.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu', padding="same"))
model_5.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu', padding="same"))
model_5.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_5.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_5.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_5.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_5.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_5.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_5.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_5.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_5.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_5.add(Layers.Conv2D(128,kernel_size=(3,3),activation='relu', padding="same"))
model_5.add(Layers.Conv2D(128,kernel_size=(3,3),activation='relu', padding="same"))
model_5.add(Layers.Conv2D(128,kernel_size=(3,3),activation='relu', padding="same"))
model_5.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))


model_5.add(Layers.Flatten())
model_5.add(Layers.Dense(128,activation='relu'))
model_5.add(Layers.Dropout(rate=0.5))
model_5.add(Layers.Dense(64,activation='relu'))
model_5.add(Layers.Dropout(rate=0.5))
model_5.add(Layers.Dense(32,activation='relu'))
model.add(Layers.Dropout(rate=0.5))
model_5.add(Layers.Dense(6,activation='softmax'))

model_5.compile(optimizer=Optimizer.Adam(lr=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model_5.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_124 (Conv2D)          (None, 150, 150, 16)      448       
_________________________________________________________________
conv2d_125 (Conv2D)          (None, 150, 150, 16)      2320      
_________________________________________________________________
conv2d_126 (Conv2D)          (None, 150, 150, 16)      2320      
_________________________________________________________________
max_pooling2d_75 (MaxPooling (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_127 (Conv2D)          (None, 75, 75, 32)        4640      
_________________________________________________________________
conv2d_128 (Conv2D)          (None, 75, 75, 32)        9248      
_________________________________________________________________
conv2d_129 (Conv2D)          (None, 75, 75, 32)      

In [56]:
run_logdir = get_run_logdir()
tensorboard_cb = Callbacks.TensorBoard(run_logdir)

history2 = model_5.fit(Images, Labels, 
                    batch_size=32, 
                    epochs=100, 
                    validation_split = 0.2,
                    callbacks=[tensorboard_cb])

Train on 11227 samples, validate on 2807 samples
Epoch 1/25
11227/11227 [==============================] - 13s 1ms/sample - loss: 1.4709 - accuracy: 0.3672 - val_loss: 1.1306 - val_accuracy: 0.5358
Epoch 2/25
11227/11227 [==============================] - 11s 1ms/sample - loss: 1.1257 - accuracy: 0.5458 - val_loss: 0.9417 - val_accuracy: 0.6145
Epoch 3/25
11227/11227 [==============================] - 11s 1ms/sample - loss: 0.9940 - accuracy: 0.6042 - val_loss: 0.8443 - val_accuracy: 0.6698
Epoch 4/25
11227/11227 [==============================] - 11s 987us/sample - loss: 0.9037 - accuracy: 0.6363 - val_loss: 0.7757 - val_accuracy: 0.6854
Epoch 5/25
11227/11227 [==============================] - 11s 979us/sample - loss: 0.8416 - accuracy: 0.6589 - val_loss: 0.8532 - val_accuracy: 0.6641
Epoch 6/25
11227/11227 [==============================] - 12s 1ms/sample - loss: 0.8166 - accuracy: 0.6786 - val_loss: 0.7653 - val_accuracy: 0.7011
Epoch 7/25
11227/11227 [=============================

In [ ]:
model_5.save('saved_model\\4x3_2mil_Relu')

In [57]:
model_6 = Models.Sequential()

model_6.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu',input_shape=(150,150,3), padding="same"))
model_6.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu', padding="same"))
model_6.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_6.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_6.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_6.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_6.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_6.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_6.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_6.add(Layers.Conv2D(128,kernel_size=(3,3),activation='relu', padding="same"))
model_6.add(Layers.Conv2D(128,kernel_size=(3,3),activation='relu', padding="same"))
model_6.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_6.add(Layers.Conv2D(256,kernel_size=(3,3),activation='relu', padding="same"))
model_6.add(Layers.Conv2D(256,kernel_size=(3,3),activation='relu', padding="same"))
model_6.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_6.add(Layers.Flatten())
model_6.add(Layers.Dense(256,activation='relu'))
model_6.add(Layers.Dropout(rate=0.5))
model_6.add(Layers.Dense(128,activation='relu'))
model_6.add(Layers.Dropout(rate=0.5))
model_6.add(Layers.Dense(64,activation='relu'))
model_6.add(Layers.Dropout(rate=0.5))
model_6.add(Layers.Dense(32,activation='relu'))
model.add(Layers.Dropout(rate=0.5))
model_6.add(Layers.Dense(6,activation='softmax'))

model_6.compile(optimizer=Optimizer.Adam(lr=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model_6.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_136 (Conv2D)          (None, 150, 150, 16)      448       
_________________________________________________________________
conv2d_137 (Conv2D)          (None, 150, 150, 16)      2320      
_________________________________________________________________
max_pooling2d_79 (MaxPooling (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_138 (Conv2D)          (None, 75, 75, 32)        4640      
_________________________________________________________________
conv2d_139 (Conv2D)          (None, 75, 75, 32)        9248      
_________________________________________________________________
max_pooling2d_80 (MaxPooling (None, 38, 38, 32)        0         
_________________________________________________________________
conv2d_140 (Conv2D)          (None, 38, 38, 64)      

In [ ]:
run_logdir = get_run_logdir()
tensorboard_cb = Callbacks.TensorBoard(run_logdir)

history2 = model_6.fit(Images, Labels, 
                    batch_size=32, 
                    epochs=100, 
                    validation_split = 0.2,
                    callbacks=[tensorboard_cb])

In [ ]:
model_6.save('saved_model\\5x2_3mil_Relu')

In [58]:
model_7 = Models.Sequential()

model_7.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu',input_shape=(150,150,3), padding="same"))
model_7.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu', padding="same"))
model_7.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu', padding="same"))
model_7.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_7.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_7.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_7.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_7.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_7.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_7.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_7.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_7.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_7.add(Layers.Conv2D(128,kernel_size=(3,3),activation='relu', padding="same"))
model_7.add(Layers.Conv2D(128,kernel_size=(3,3),activation='relu', padding="same"))
model_7.add(Layers.Conv2D(128,kernel_size=(3,3),activation='relu', padding="same"))
model_7.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_7.add(Layers.Conv2D(256,kernel_size=(3,3),activation='relu', padding="same"))
model_7.add(Layers.Conv2D(256,kernel_size=(3,3),activation='relu', padding="same"))
model_7.add(Layers.Conv2D(256,kernel_size=(3,3),activation='relu', padding="same"))
model_7.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_7.add(Layers.Flatten())
model_7.add(Layers.Dense(256,activation='relu'))
model_7.add(Layers.Dropout(rate=0.5))
model_7.add(Layers.Dense(128,activation='relu'))
model_7.add(Layers.Dropout(rate=0.5))
model_7.add(Layers.Dense(64,activation='relu'))
model_7.add(Layers.Dropout(rate=0.5))
model_7.add(Layers.Dense(32,activation='relu'))
model.add(Layers.Dropout(rate=0.5))
model_7.add(Layers.Dense(6,activation='softmax'))

model_7.compile(optimizer=Optimizer.Adam(lr=0.0001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model_7.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_146 (Conv2D)          (None, 150, 150, 16)      448       
_________________________________________________________________
conv2d_147 (Conv2D)          (None, 150, 150, 16)      2320      
_________________________________________________________________
conv2d_148 (Conv2D)          (None, 150, 150, 16)      2320      
_________________________________________________________________
max_pooling2d_84 (MaxPooling (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_149 (Conv2D)          (None, 75, 75, 32)        4640      
_________________________________________________________________
conv2d_150 (Conv2D)          (None, 75, 75, 32)        9248      
_________________________________________________________________
conv2d_151 (Conv2D)          (None, 75, 75, 32)      

In [ ]:
run_logdir = get_run_logdir()
tensorboard_cb = Callbacks.TensorBoard(run_logdir)

history2 = model_7.fit(Images, Labels, 
                    batch_size=32, 
                    epochs=100, 
                    validation_split = 0.2,
                    callbacks=[tensorboard_cb])

In [ ]:
model_7.save('saved_model\\5x3_3mil_Relu')

In [11]:
model_8 = Models.Sequential()

model_8.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu',input_shape=(150,150,3), padding="same"))
model_8.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu', padding="same"))
model_8.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_8.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_8.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_8.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_8.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_8.add(Layers.Conv2D(128,kernel_size=(3,3),activation='relu', padding="same"))
model_8.add(Layers.Conv2D(128,kernel_size=(3,3),activation='relu', padding="same"))
model_8.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_8.add(Layers.Conv2D(256,kernel_size=(3,3),activation='relu', padding="same"))
model_8.add(Layers.Conv2D(256,kernel_size=(3,3),activation='relu', padding="same"))
model_8.add(Layers.Conv2D(384,kernel_size=(3,3),activation='relu', padding="same"))
model_8.add(Layers.Conv2D(384,kernel_size=(3,3),activation='relu', padding="same"))
model_8.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_8.add(Layers.Conv2D(512,kernel_size=(3,3),activation='relu', padding="same"))
model_8.add(Layers.Conv2D(512,kernel_size=(3,3),activation='relu', padding="same"))
model_8.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_8.add(Layers.Flatten())

model_8.add(Layers.Dense(512,activation='relu'))
model_8.add(Layers.Dropout(rate=0.5))

model_8.add(Layers.Dense(384,activation='relu'))
model_8.add(Layers.Dropout(rate=0.5))

model_8.add(Layers.Dense(256,activation='relu'))
model_8.add(Layers.Dropout(rate=0.5))

model_8.add(Layers.Dense(128,activation='relu'))
model_8.add(Layers.Dropout(rate=0.5))

model_8.add(Layers.Dense(64,activation='relu'))
model_8.add(Layers.Dropout(rate=0.5))

model_8.add(Layers.Dense(32,activation='relu'))
model_8.add(Layers.Dropout(rate=0.5))

model_8.add(Layers.Dense(6,activation='softmax'))

model_8.compile(optimizer=Optimizer.Adam(lr=0.00001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model_8.summary()
'''
Largest convolution model, 19Mil params
looks like it starts over fitting around 75 epochs, but valadation accuracy continued to increase
'''

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_43 (Conv2D)           (None, 150, 150, 16)      448       
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 150, 150, 16)      2320      
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 150, 150, 32)      4640      
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 150, 150, 32)      9248      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 75, 75, 32)        0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 75, 75, 64)        18496     
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 75, 75, 64)       

'\nLargest convolution model, 19Mil params\nlooks like it starts over fitting around 75 epochs, but valadation accuracy continued to increase\n'

In [12]:
run_logdir = get_run_logdir()
tensorboard_cb = Callbacks.TensorBoard(run_logdir)

history2 = model_8.fit(Images, Labels, 
                    batch_size=32, 
                    epochs=200, 
                    validation_split = 0.2,
                    callbacks=[tensorboard_cb])

Train on 11227 samples, validate on 2807 samples
Epoch 1/200
 4800/11227 [===========>..................] - ETA: 40s - loss: 1.8275 - accuracy: 0.1818

KeyboardInterrupt: 

In [ ]:
model_8.save('saved_model\\4xMixed_34mil_Relu')

In [6]:
model_9 = Models.Sequential()

model_9.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu',input_shape=(150,150,3), padding="same"))
model_9.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_9.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_9.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_9.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_9.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_9.add(Layers.Conv2D(128,kernel_size=(3,3),activation='relu', padding="same"))
model_9.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_9.add(Layers.Conv2D(256,kernel_size=(3,3),activation='relu', padding="same"))
model_9.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_9.add(Layers.Conv2D(384,kernel_size=(3,3),activation='relu', padding="same"))
model_9.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_9.add(Layers.Conv2D(512,kernel_size=(3,3),activation='relu', padding="same"))
model_9.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_9.add(Layers.Flatten())

model_9.add(Layers.Dense(512,activation='relu'))
model_9.add(Layers.Dropout(rate=0.5))
model_9.add(Layers.Dense(384,activation='relu'))
model_9.add(Layers.Dropout(rate=0.5))
model_9.add(Layers.Dense(256,activation='relu'))
model_9.add(Layers.Dropout(rate=0.5))
model_9.add(Layers.Dense(128,activation='relu'))
model_9.add(Layers.Dropout(rate=0.5))
model_9.add(Layers.Dense(64,activation='relu'))
model_9.add(Layers.Dropout(rate=0.5))
model_9.add(Layers.Dense(32,activation='relu'))
model_9.add(Layers.Dropout(rate=0.5))

model_9.add(Layers.Dense(6,activation='softmax'))

model_9.compile(optimizer=Optimizer.Adam(lr=0.00001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model_9.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_36 (Conv2D)           (None, 150, 150, 16)      448       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 75, 75, 16)        0         
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 75, 75, 32)        4640      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 38, 38, 32)        0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 38, 38, 64)        18496     
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 19, 19, 64)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 19, 19, 128)      

In [64]:
run_logdir = get_run_logdir()
tensorboard_cb = Callbacks.TensorBoard(run_logdir)

history2 = model_9.fit(Images, Labels, 
                    batch_size=32, 
                    epochs=200, 
                    validation_split = 0.2,
                    callbacks=[tensorboard_cb])

Train on 11227 samples, validate on 2807 samples
Epoch 1/25
11227/11227 [==============================] - 7s 662us/sample - loss: 2.7197 - accuracy: 0.1634 - val_loss: 1.7834 - val_accuracy: 0.2284
Epoch 2/25
11227/11227 [==============================] - 6s 565us/sample - loss: 1.8174 - accuracy: 0.1662 - val_loss: 1.7882 - val_accuracy: 0.2309
Epoch 3/25
11227/11227 [==============================] - 7s 580us/sample - loss: 1.8010 - accuracy: 0.1703 - val_loss: 1.7854 - val_accuracy: 0.2408
Epoch 4/25
11227/11227 [==============================] - 6s 565us/sample - loss: 1.7954 - accuracy: 0.1724 - val_loss: 1.7895 - val_accuracy: 0.2145
Epoch 5/25
11227/11227 [==============================] - 6s 572us/sample - loss: 1.7923 - accuracy: 0.1726 - val_loss: 1.7879 - val_accuracy: 0.2291
Epoch 6/25
11227/11227 [==============================] - 6s 565us/sample - loss: 1.7913 - accuracy: 0.1802 - val_loss: 1.7896 - val_accuracy: 0.2180
Epoch 7/25
11227/11227 [===========================

In [ ]:
model_9.save('saved_model\\7x1_4mil_Relu')

In [ ]:
# !tensorboard --logdir=./my_logs --port=6006

In [5]:
model_10 = Models.Sequential()

model_10.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu',input_shape=(150,150,3), padding="same"))
model_10.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_10.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_10.add(Layers.Conv2D(128,kernel_size=(3,3),activation='relu', padding="same"))
model_10.add(Layers.Conv2D(256,kernel_size=(3,3),activation='relu', padding="same"))
model_10.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_10.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu', padding="same"))
model_10.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_10.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_10.add(Layers.Conv2D(128,kernel_size=(3,3),activation='relu', padding="same"))
model_10.add(Layers.Conv2D(256,kernel_size=(3,3),activation='relu', padding="same"))
model_10.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_10.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu', padding="same"))
model_10.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_10.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_10.add(Layers.Conv2D(128,kernel_size=(3,3),activation='relu', padding="same"))
model_10.add(Layers.Conv2D(256,kernel_size=(3,3),activation='relu', padding="same"))
model_10.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_10.add(Layers.Conv2D(16,kernel_size=(3,3),activation='relu', padding="same"))
model_10.add(Layers.Conv2D(32,kernel_size=(3,3),activation='relu', padding="same"))
model_10.add(Layers.Conv2D(64,kernel_size=(3,3),activation='relu', padding="same"))
model_10.add(Layers.Conv2D(128,kernel_size=(3,3),activation='relu', padding="same"))
model_10.add(Layers.Conv2D(256,kernel_size=(3,3),activation='relu', padding="same"))
model_10.add(Layers.MaxPool2D((2,2), strides=2, padding="same"))

model_10.add(Layers.Flatten())

model_10.add(Layers.Dense(256,activation='relu'))
model_10.add(Layers.Dropout(rate=0.5))
model_10.add(Layers.Dense(128,activation='relu'))
model_10.add(Layers.Dropout(rate=0.5))
model_10.add(Layers.Dense(64,activation='relu'))
model_10.add(Layers.Dropout(rate=0.5))
model_10.add(Layers.Dense(32,activation='relu'))
model_10.add(Layers.Dropout(rate=0.5))

model_10.add(Layers.Dense(6,activation='softmax'))

model_10.compile(optimizer=Optimizer.Adam(lr=0.00001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model_10.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 150, 150, 16)      448       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 150, 150, 32)      4640      
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 150, 150, 64)      18496     
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 150, 150, 128)     73856     
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 150, 150, 256)     295168    
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 75, 75, 256)       0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 75, 75, 16)       

In [ ]:
run_logdir = get_run_logdir()
tensorboard_cb = Callbacks.TensorBoard(run_logdir)

history2 = model_10.fit(Images, Labels, 
                    batch_size=32, 
                    epochs=200, 
                    validation_split = 0.2,
                    callbacks=[tensorboard_cb])

In [ ]:
model_10.save('saved_model\\4x5mix_8mil_Relu')